In [95]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"

In [96]:
!pip install datasets trl bitsandbytes accelerate rouge_score evaluate

In [97]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np

In [98]:
# Global seed definition for reproducibility
GLOBAL_SEED = 42

# Set seeds for all relevant libraries
import numpy as np
import torch
import random
from transformers import set_seed

# Set seeds for Python's random module
random.seed(GLOBAL_SEED)

# Set seeds for NumPy
np.random.seed(GLOBAL_SEED)

# Set seeds for PyTorch
torch.manual_seed(GLOBAL_SEED)
torch.cuda.manual_seed_all(GLOBAL_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Set seeds for Transformers
set_seed(GLOBAL_SEED)

print(f"All random seeds have been set to {GLOBAL_SEED} for reproducibility.")

All random seeds have been set to 42 for reproducibility.


In [99]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [100]:
# https://huggingface.co/datasets/abisee/cnn_dailymail
huggingface_dataset_name = "abisee/cnn_dailymail"
dataset = load_dataset(huggingface_dataset_name, "3.0.0")  # Using version 3.0.0 (non-anonymized)
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [101]:
dataset['train'][0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [102]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )
device_map = {"": 0}

In [103]:
model_name='microsoft/phi-2'
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [104]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [105]:
print_gpu_utilization()

GPU memory occupied: 9196 MB.


In [106]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_name, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model, p, maxlen=100, sample=True, seed=GLOBAL_SEED):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(
        **toks.to("cuda"), 
        max_new_tokens=maxlen, 
        do_sample=sample,
        num_return_sequences=1,
        temperature=0.1,
        num_beams=1,
        top_p=0.95,
    ).to('cpu')
    return eval_tokenizer.batch_decode(res, skip_special_tokens=True)

In [107]:
%%time
index = 10

article = dataset['test'][index]['article']
summary = dataset['test'][index]['highlights']

formatted_prompt = f"Instruct: Summarize the following article.\n{article}\nOutput:\n"
res = gen(original_model, formatted_prompt, 100,)
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Summarize the following article.
London (CNN)A 19-year-old man was charged Wednesday with terror offenses after he was arrested as he returned to Britain from Turkey, London's Metropolitan Police said. Yahya Rashid, a UK national from northwest London, was detained at Luton airport on Tuesday after he arrived on a flight from Istanbul, police said. He's been charged with engaging in conduct in preparation of acts of terrorism, and with engaging in conduct with the intention of assisting others to commit acts of terrorism. Both charges relate to the period between November 1 and March 31. Rashid is due to appear in Westminster Magistrates' Court on Wednesday, police said. CNN's Lindsay Isaac contributed to this report.
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
London's Metrop

In [108]:
# filepath: /Users/madnanrizqu/Code/KAU/cs681-final/summarization/main.ipynb
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Summarize the below article."
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['article']}" if sample["article"] else None
    response = f"{RESPONSE_KEY}\n{sample['highlights']}"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

In [109]:
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

In [110]:
# Combine all splits into one dataset
combined_dataset = {
    'train': dataset['train'],
    'validation': dataset['validation'],
    'test': dataset['test']
}

# Get the total size
train_size = len(combined_dataset['train'])
validation_size = len(combined_dataset['validation'])
test_size = len(combined_dataset['test'])
total_size = train_size + validation_size + test_size

print(f"Total dataset size: {total_size}")

Total dataset size: 311971


In [111]:
# Calculate new split sizes
new_train_size = int(0.70 * total_size)
new_val_size = int(0.15 * total_size)
new_dev_size = int(0.10 * total_size)
new_test_size = int(0.05 * total_size)

print(f"New train size: {new_train_size}")
print(f"New validation size: {new_val_size}")
print(f"New dev size: {new_dev_size}")
print(f"New test size: {new_test_size}")

New train size: 218379
New validation size: 46795
New dev size: 31197
New test size: 15598


In [112]:
from datasets import concatenate_datasets

# Concatenate all splits into a single dataset
all_data = concatenate_datasets([
    dataset['train'],
    dataset['validation'],
    dataset['test']
])

# Ensure deterministic shuffling using GLOBAL_SEED
shuffled_indices = np.random.permutation(len(all_data))
all_data = all_data.select(shuffled_indices)

# Create new splits
new_train_size = int(0.70 * len(all_data))
new_val_size = int(0.15 * len(all_data))
new_dev_size = int(0.10 * len(all_data))
new_test_size = int(0.05 * len(all_data))

# Create new splits
new_train_dataset = all_data.select(range(0, new_train_size))
new_val_dataset = all_data.select(range(new_train_size, new_train_size + new_val_size))
new_dev_dataset = all_data.select(range(new_train_size + new_val_size, new_train_size + new_val_size + new_dev_size))
new_test_dataset = all_data.select(range(new_train_size + new_val_size + new_dev_size, new_train_size + new_val_size + new_dev_size + new_test_size))

print(f"New train dataset size: {len(new_train_dataset)}")
print(f"New validation dataset size: {len(new_val_dataset)}")
print(f"New dev dataset size: {len(new_dev_dataset)}")
print(f"New test dataset size: {len(new_test_dataset)}")

New train dataset size: 218379
New validation dataset size: 46795
New dev dataset size: 31197
New test dataset size: 15598


In [113]:
from functools import partial

def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)

    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['id', 'article', 'highlights'],  # Updated column names
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [114]:
print_gpu_utilization()

GPU memory occupied: 9266 MB.


In [115]:
# Preprocess all splits
max_length = get_max_length(original_model)
print(f"Using max length: {max_length}")

# Process each split using the global seed
train_dataset = preprocess_dataset(tokenizer, max_length, GLOBAL_SEED, new_train_dataset.select(range(210)))
eval_dataset = preprocess_dataset(tokenizer, max_length, GLOBAL_SEED, new_val_dataset.select(range(45)))
dev_dataset = preprocess_dataset(tokenizer, max_length, GLOBAL_SEED, new_dev_dataset.select(range(30)))
test_dataset = preprocess_dataset(tokenizer, max_length, GLOBAL_SEED, new_test_dataset.select(range(15)))

# Print dataset shapes
print(f"Shapes of the datasets:")
print(f"Training: {train_dataset.shape}")
print(f"Validation: {eval_dataset.shape}")
print(f"Dev: {dev_dataset.shape}")
print(f"Test: {test_dataset.shape}")

Found max lenth: 2048
Using max length: 2048
Preprocessing dataset...
Preprocessing dataset...
Preprocessing dataset...
Preprocessing dataset...
Shapes of the datasets:
Training: (203, 3)
Validation: (44, 3)
Dev: (28, 3)
Test: (15, 3)


In [116]:
print(f"Shapes of the datasets:")
print(f"Training: {train_dataset.shape}")
print(f"Validation: {eval_dataset.shape}")
print(train_dataset)

Shapes of the datasets:
Training: (203, 3)
Validation: (44, 3)
Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 203
})


In [117]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 262364160
all model parameters: 1521392640
percentage of trainable model parameters: 17.24%


In [118]:
print(original_model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_layernorm): 

In [119]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

# 2 - Using the prepare_model_for_kbit_training method from PEFT
original_model = prepare_model_for_kbit_training(original_model)

peft_model = get_peft_model(original_model, config)

In [120]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 20971520
all model parameters: 1542364160
percentage of trainable model parameters: 1.36%


In [121]:
print(peft_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4b

In [122]:
output_dir = './peft-dialogue-summary-training/final-checkpoint'
import transformers

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=10,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=10,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=10,
    eval_strategy="steps",
    eval_steps=10,
    num_train_epochs=1,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
    seed=GLOBAL_SEED,  # Add global seed
    data_seed=GLOBAL_SEED  # Add data seed for data loaders
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [123]:
peft_training_args.device

device(type='cuda', index=0)

In [124]:
peft_trainer.train()

Step,Training Loss,Validation Loss
10,2.437600,2.414258


TrainOutput(global_step=10, training_loss=2.4376129150390624, metrics={'train_runtime': 214.3319, 'train_samples_per_second': 0.187, 'train_steps_per_second': 0.047, 'total_flos': 667241004165120.0, 'train_loss': 2.4376129150390624, 'epoch': 0.19704433497536947})

In [125]:
print_gpu_utilization()

GPU memory occupied: 14952 MB.


In [126]:
# Step 8: Evaluate on Dev Split
print("Evaluating on Dev split...")
dev_results = peft_trainer.evaluate(dev_dataset)
print(f"Dev set evaluation results:")
print(dev_results)

Evaluating on Dev split...


Dev set evaluation results:
{'eval_loss': 2.339982748031616, 'eval_runtime': 43.8914, 'eval_samples_per_second': 0.638, 'eval_steps_per_second': 0.091, 'epoch': 0.19704433497536947}


In [127]:
# Step 9: Final Evaluation on Test Split (only run this once training is complete)
print("Performing final evaluation on Test split...")
test_results = peft_trainer.evaluate(test_dataset)
print(f"Final test set evaluation results:")
print(test_results)

Performing final evaluation on Test split...
Final test set evaluation results:
{'eval_loss': 2.446943998336792, 'eval_runtime': 31.0249, 'eval_samples_per_second': 0.483, 'eval_steps_per_second': 0.064, 'epoch': 0.19704433497536947}


In [128]:
# Free memory for merging weights
if peft_trainer:
  del peft_trainer

torch.cuda.empty_cache()

In [129]:
print_gpu_utilization()

GPU memory occupied: 5976 MB.


In [130]:
import pandas as pd
import evaluate
import torch
import numpy as np

articles = new_test_dataset[15:25]['article']
human_baseline_summaries = new_test_dataset[15:25]['highlights']

original_model_summaries = []
peft_model_summaries = []

for idx, article in enumerate(articles):
    human_baseline_text_output = human_baseline_summaries[idx]
    
    # Use prompt format consistent with training
    prompt = f"\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruct: Summarize the below article.\n\n{article}\n\n### Output:\n"

    # Increase token limit
    original_model_res = gen(original_model, prompt, 150)
    
    # More robust parsing with error handling
    try:
        if '### Output:' in original_model_res[0]:
            original_model_text_output = original_model_res[0].split('### Output:')[1].strip()
        elif 'Output:' in original_model_res[0]:
            original_model_text_output = original_model_res[0].split('Output:')[1].strip()
        else:
            original_model_text_output = original_model_res[0]
    except Exception as e:
        print(f"Error parsing original model output for article {idx}: {e}")
        original_model_text_output = "[Parsing error]"

    # Same for PEFT model
    peft_model_res = gen(peft_model, prompt, 150)
    
    try:
        if '### Output:' in peft_model_res[0]:
            peft_model_output = peft_model_res[0].split('### Output:')[1].strip()
        elif 'Output:' in peft_model_res[0]:
            peft_model_output = peft_model_res[0].split('Output:')[1].strip()
        else:
            peft_model_output = peft_model_res[0]
            
        # Handle various endings
        for marker in ['### End', '#End', '##OUTPUT', '##End']:
            if marker in peft_model_output:
                peft_model_text_output = peft_model_output.split(marker)[0].strip()
                break
        else:  # No marker found
            peft_model_text_output = peft_model_output.strip()
            
    except Exception as e:
        print(f"Error parsing PEFT model output for article {idx}: {e}")
        peft_model_text_output = "[Parsing error]"

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])
df

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

,human_baseline_summaries,original_model_summaries,peft_model_summaries
0,France announces new ministers .\nThe governme...,"A new French government was announced Tuesday,...",A new French government was announced on Tuesd...
1,Miss California USA Carrie Prejean says she po...,A second lingerie-modeling photo of Miss Calif...,A second lingerie-modeling photo of Miss Calif...
2,Jay-Z reported to be 'deeply disappointed' by ...,Rihanna was pictured out in London today looki...,Rihanna was pictured out in London today looki...
3,Carbisdale Castle near Culrain is nestled on p...,"Carbisdale Castle, near the village of Culrain...","Carbisdale Castle, near the Highland village o..."
4,Police collated the four-page dossier followin...,An unnamed hit band from the 1960s attended th...,An unnamed hit band from the 1960s attended th...
5,She still owns £3.9 million stately home in Ea...,Heather Mills has sold her British beachfront ...,Heather Mills has sold her British beachfront ...
6,Mitt Romney is trying to outdo President Obama...,The article is about how Mitt Romney is trying...,The article is about how Mitt Romney is trying...
7,Archaeologists believe site was centre of a ne...,The article describes how archaeologists have ...,The article discusses the latest discovery of ...
8,Edward Snowden claimed that a damaging culture...,"Edward Snowden, who worked as a contract emplo...","Edward Snowden, who worked as a contract emplo..."
9,Enterovirus D68 is sending children with breat...,Enterovirus D68 has swept through 30 states si...,Enterovirus D68 has swept through 30 states si...


In [131]:
import evaluate
# ROUGE evaluation
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# BLEU evaluation
bleu = evaluate.load('bleu')

original_model_bleu = bleu.compute(
    predictions=original_model_summaries,
    references=[[ref] for ref in human_baseline_summaries]
)

peft_model_bleu = bleu.compute(
    predictions=peft_model_summaries, 
    references=[[ref] for ref in human_baseline_summaries]
)

# Fixed perplexity calculation function
def calculate_perplexity_alt(model, tokenizer, texts, max_loss=20.0):
    perplexities = []
    
    with torch.no_grad():
        for text in texts:
            try:
                # Tokenize without setting labels
                inputs = tokenizer(text, return_tensors="pt")
                # Move to device
                inputs = {k: v.to(model.device) for k, v in inputs.items()}
                
                # Forward pass to get logits
                outputs = model(**inputs)
                logits = outputs.logits
                
                # Calculate loss manually
                shift_logits = logits[..., :-1, :].contiguous()
                shift_labels = inputs['input_ids'][..., 1:].contiguous()
                
                loss_fct = torch.nn.CrossEntropyLoss(reduction='mean')
                loss = loss_fct(shift_logits.reshape(-1, shift_logits.size(-1)), 
                              shift_labels.reshape(-1))
                
                # Clip loss to prevent overflow
                loss = torch.clamp(loss, 0, max_loss)
                
                # Calculate perplexity
                perplexity = torch.exp(loss)
                
                if not torch.isnan(perplexity) and not torch.isinf(perplexity):
                    perplexities.append(perplexity.item())
            except Exception as e:
                print(f"Error processing text: {e}")
                continue
    
    if not perplexities:
        return float('nan')
    
    return sum(perplexities) / len(perplexities)

# Try the alternative calculation
original_perplexity = calculate_perplexity_alt(original_model, eval_tokenizer, human_baseline_summaries)
peft_perplexity = calculate_perplexity_alt(peft_model, eval_tokenizer, peft_model_summaries)

# Helper function for formatting outputs
def format_metrics(rouge_results, bleu_results, perplexity):
    print("=== ROUGE Scores ===")
    for metric, value in rouge_results.items():
        print(f"  {metric}: {value*100:.2f}%")
    
    print("\n=== BLEU Score ===")
    print(f"  BLEU: {bleu_results['bleu']*100:.2f}%")
    print("  Precision by n-gram:")
    for n, precision in enumerate(bleu_results['precisions'], 1):
        print(f"    {n}-gram: {precision*100:.2f}%")
    
    print(f"\n=== Perplexity ===")
    print(f"  {perplexity:.4f} (lower is better)")

# Print results with better formatting
print("\n" + "="*50)
print("ORIGINAL MODEL EVALUATION")
print("="*50)
format_metrics(original_model_results, original_model_bleu, original_perplexity)

print("\n\n" + "="*50)
print("PEFT MODEL EVALUATION")
print("="*50)
format_metrics(peft_model_results, peft_model_bleu, peft_perplexity)

# Calculate and display improvements
print("\n\n" + "="*50)
print("IMPROVEMENT SUMMARY")
print("="*50)

# ROUGE improvement
print("=== ROUGE Improvement ===")
for metric in original_model_results.keys():
    improvement = (peft_model_results[metric] - original_model_results[metric]) * 100
    print(f"  {metric}: {improvement:.2f}% absolute improvement")

# BLEU improvement
bleu_improvement = (peft_model_bleu['bleu'] - original_model_bleu['bleu']) * 100
print("\n=== BLEU Improvement ===")
print(f"  {bleu_improvement:.2f}% absolute improvement")

# Perplexity improvement (lower is better)
perplexity_improvement = original_perplexity - peft_perplexity
perplexity_improvement_percent = (perplexity_improvement / original_perplexity) * 100
print("\n=== Perplexity Improvement ===")
print(f"  {perplexity_improvement_percent:.2f}% reduction")


ORIGINAL MODEL EVALUATION
=== ROUGE Scores ===
  rouge1: 31.80%
  rouge2: 9.84%
  rougeL: 19.20%
  rougeLsum: 23.78%

=== BLEU Score ===
  BLEU: 4.67%
  Precision by n-gram:
    1-gram: 22.34%
    2-gram: 6.16%
    3-gram: 2.87%
    4-gram: 1.20%

=== Perplexity ===
  44.5138 (lower is better)


PEFT MODEL EVALUATION
=== ROUGE Scores ===
  rouge1: 31.32%
  rouge2: 10.37%
  rougeL: 19.25%
  rougeLsum: 24.23%

=== BLEU Score ===
  BLEU: 4.68%
  Precision by n-gram:
    1-gram: 21.94%
    2-gram: 6.24%
    3-gram: 2.80%
    4-gram: 1.25%

=== Perplexity ===
  11.9945 (lower is better)


IMPROVEMENT SUMMARY
=== ROUGE Improvement ===
  rouge1: -0.48% absolute improvement
  rouge2: 0.54% absolute improvement
  rougeL: 0.05% absolute improvement
  rougeLsum: 0.45% absolute improvement

=== BLEU Improvement ===
  0.01% absolute improvement

=== Perplexity Improvement ===
  73.05% reduction
